# 라벨링된 채널 정보 -> 비디오 100개씩 추출 및 라벨링 적용

In [3]:
from youtube_crawler import youtubeCrawler
import json
import pandas as pd

with open('APIkey.json') as f:
    key = json.load(f);
    apikey = key['APIkey'];


YC = youtubeCrawler(apikey)

channel_df = pd.read_csv('learning_data/channel_labeled.csv')
channel_df

video_labeled_df = pd.DataFrame(columns=['video_id', 'difficulty', 'channel_id'])


for idx, channelID in enumerate(channel_df['channel_id']):
    # if idx != 6: continue
    difficulty = channel_df['difficulty'][idx]
    try:
        new_videos = YC.get_recent_videos(channelID)
        for video_id in new_videos:
            video_labeled_df.loc[len(video_labeled_df)] = [video_id, difficulty, channelID]
        
    except:
        print('something wrong')
        break
    print(f'{idx} {channelID} OK')
    

video_labeled_df.to_csv('learning_data/video_labeled.csv', sep=',', index=False)

    

UCWCOoCApFUtccYENOYeL1uw vidoes OK
0 UCWCOoCApFUtccYENOYeL1uw OK
UCqa2MPu8bLY1PwVFUpSyVhQ vidoes OK
1 UCqa2MPu8bLY1PwVFUpSyVhQ OK
UCAo41QETLlwvJ-oqjmEPt8A vidoes OK
2 UCAo41QETLlwvJ-oqjmEPt8A OK
no more videos
3 UCAOtE1V7Ots4DjM8JLlrYgg OK
no more videos
4 UCoookXUzPciGrEZEXmh4Jjg OK
no more videos
UCLbyJSPEe2lx3jY-G0CFR3g vidoes OK
5 UCLbyJSPEe2lx3jY-G0CFR3g OK
no more videos
6 UCDCNmuaOXOo25Yn4mbMHhhQ OK
UCFuU-5B1eKAWaTeLUu3JuyA vidoes OK
7 UCFuU-5B1eKAWaTeLUu3JuyA OK
no more videos
8 UCcdwLMPsaU2ezNSJU1nFoBQ OK
no more videos
UCbCmjCuTUZos6Inko4u57UQ vidoes OK
9 UCbCmjCuTUZos6Inko4u57UQ OK
no more videos
10 UCoL0M9swO14BT8u9pTn9MvQ OK
no more videos
11 UCE_URvMQZgvPqd8rzlFF58A OK
no more videos
12 UC5PYHgAzJ1wLEidB58SK6Xw OK
UCvlE5gTbOvjiolFlEm-c_Ow vidoes OK
13 UCvlE5gTbOvjiolFlEm-c_Ow OK
no more videos
14 UCH2cMLriCDcvb9Uq_2kf17A OK
no more videos
15 UCUsN5ZwHx2kILm84-jPDeXw OK
no more videos
16 UCRijo3ddMTht_IHyNSNXpNQ OK
UCa6vGFO9ty8v5KZJXQxdhaw vidoes OK
17 UCa6vGFO9ty8v5KZJXQx

# 분석기를 통해 비디오 스크립트 분석 

 punctuator 가 머신러닝을 통해 작동하므로 적절한 환경에서 실행해야 오래 걸리지 않음

In [4]:
import pandas as pd
from core import analyzeAll

import json

no_caption_videos =[]
train_data = pd.DataFrame(columns=['videoId','totalWords','totalUniqueWords','totalSentences','avgSyllPerSec','avgCEFRScore','avgWordCEFR','avgFreqCEFR','readability','avgSentenceLength','uncommonRatio','totalEasyRatio','totalMiddleRatio','totalHardRatio','wordEasyRatio','wordMiddleRatio','wordHardRatio','FreqEasyRatio','FreqMiddleRatio','FreqHardRatio','channelId'])

video_labeled = pd.read_csv('learning_data/video_labeled.csv')
for idx,video_id in enumerate(video_labeled['video_id']):
    if idx > -1:
        print(f'{idx} {video_id} start')
        try:
            result = analyzeAll(video_id)
            no_script_result = json.loads(result)
            del no_script_result['script']
            train_data.loc[len(train_data)] = no_script_result
            channel_id = video_labeled['channel_id'][idx]
            print(channel_id)
            train_data.loc[len(train_data)-1, 'channelId'] = channel_id
        except NotImplementedError:
            print(f'{idx} {video_id} no captions')
            no_caption_videos.append(video_id)
        if idx % 300 == 0:
            train_data.to_csv('learning_data/video_train_upgrade.csv',sep=',',index=False)
            print("####temp download#########")
        


train_data.to_csv('learning_data/video_train_upgrade.csv',sep=',',index=False)
print(no_caption_videos)

LookupError: 
**********************************************************************
  Resource [93mwordnet[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('wordnet')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/wordnet[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


# 난이도 라벨과 분석 결과 합치기

In [ ]:
import pandas as pd

data0 = pd.read_csv('learning_data/video_train.csv', sep=',')
data1 = pd.read_csv('learning_data/video_train_1.csv', sep=',')
data2 = pd.read_csv('learning_data/video_train_2.csv', sep=',')
data3 = pd.read_csv('learning_data/video_train_3.csv', sep=',')


merge = pd.concat([data0,data1,data2,data3], ignore_index=True)

label = pd.read_csv('learning_data/video_labeled.csv', sep=',')

result = pd.merge(merge, label, how='left', left_on='videoId', right_on='video_id')
# result.drop(['channel_id', 'video_id'], axis=1, inplace=True)
result.to_csv('learning_data/capstone_train_data.csv', sep=',', index=False)
